In [9]:
import pickle
import sqlite3
import pandas as pd

In [11]:
pickle_file = 'df_bias.pkl'  

with open(pickle_file, 'rb') as f:
    data = pickle.load(f)

# Preprocessing for df_topic_and_loc.pkl
# data['date_published'] = data.date_published.astype(str)
# data = data.drop('topic_list', axis='columns')
# data.sort_values(by='date_published')
data

,article_url,bias_rating,generalisation,prominence,negative_behaviour,misrepresentation,headline_or_imagery
0,https://www.dailymail.co.uk/wires/reuters/arti...,-1,0,0,1,0,0
1,https://www.theguardian.com/world/2019/jun/13/...,-1,0,0,1,0,0
2,https://www.thejc.com/news/us-news/us-army-vet...,0,0,0,0,0,0
3,https://www.express.co.uk/news/politics/115003...,-1,0,0,1,0,0
4,https://www.christiantoday.com/article/in-pack...,0,0,0,0,0,0
...,...,...,...,...,...,...,...
495,https://www.dailymail.co.uk/wires/ap/article-7...,0,0,0,0,0,0
496,https://www.dailymail.co.uk/news/article-67644...,0,0,0,0,0,0
497,https://www.bbc.co.uk/news/world-asia-india-49...,0,0,0,0,0,0
498,https://www.dailymail.co.uk/wires/ap/article-7...,1,0,0,1,1,0


In [14]:
data.generalisation.describe()

count    500.000000
mean       0.056000
std        0.230152
min        0.000000
25%        0.000000
50%        0.000000
75%        0.000000
max        1.000000
Name: generalisation, dtype: float64

In [20]:
cursor.execute('DROP TABLE bias_rating')
conn.commit()
conn.close()

In [21]:
db_path = 'cfmm.db'
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

table_name = 'bias_rating'
create_table_query = f"""
CREATE TABLE IF NOT EXISTS {table_name} (
    article_url TEXT,
    bias_rating INTEGER,
    generalisation INTEGER,
    prominence INTEGER,
    negative_behaviour INTEGER,
    misrepresentation INTEGER,
    headline_or_imagery INTEGER
);
"""

cursor.execute(create_table_query)

# 5. Prepare an SQL INSERT query
insert_query = f"""
INSERT INTO {table_name} (
    article_url,
    bias_rating,
    generalisation,
    prominence,
    negative_behaviour,
    misrepresentation, 
    headline_or_imagery
) VALUES (?, ?, ?, ?, ?, ?, ?)
"""

# 6. Iterate through the DataFrame and insert each row
for _, row in data.iterrows():
    # Convert the row to a tuple of values
    values = tuple(row[col] for col in data.columns)
    cursor.execute(insert_query, values)

# 6. Commit and close the connection
conn.commit()
conn.close()


In [22]:
db_path = 'cfmm.db'
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

table_name = 'articles'
create_table_query = f"""
CREATE TABLE IF NOT EXISTS {table_name} (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    date_published TEXT,
    publisher TEXT,
    title TEXT,
    text TEXT,
    article_url TEXT,
    topic TEXT,
    location TEXT,
    main_site TEXT
);
"""

cursor.execute(create_table_query)

# 5. Prepare an SQL INSERT query
insert_query = f"""
INSERT INTO {table_name} (
    date_published, publisher, title, text, article_url, 
    topic, location, main_site
) VALUES (?, ?, ?, ?, ?, ?, ?, ?)
"""

# 6. Iterate through the DataFrame and insert each row
for _, row in data.iterrows():
    # Convert the row to a tuple of values
    values = tuple(row[col] for col in data.columns)
    cursor.execute(insert_query, values)

# 6. Commit and close the connection
conn.commit()
conn.close()

In [24]:
db_path = 'cfmm.db'
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

sql_query = 'SELECT * FROM articles a INNER JOIN bias_rating b ON a.article_url = b.article_url'

conn = sqlite3.connect(db_path)
pd.read_sql_query(sql_query, conn)

,id,date_published,publisher,title,text,article_url,topic,topic_list,location,main_site,article_url,bias_rating,generalisation,prominence,negative_behaviour,misrepresentation,headline_or_imagery
0,1,2019-05-23,Dailymail,Iran says it will not surrender even if it is ...,"LONDON, May 23 (Reuters) - Iranian President H...",https://www.dailymail.co.uk/wires/reuters/arti...,Crimes and Arrests | Politics,None,"Iran, Islamic Republic of",www.dailymail.co.uk,https://www.dailymail.co.uk/wires/reuters/arti...,-1,0,0,1,0,0
1,2,2019-06-13,Theguardian,MEPs create biggest far-right group in Europea...,"Geert Wilders, left, Matteo Salvini, centre, a...",https://www.theguardian.com/world/2019/jun/13/...,Politics,None,Italy,www.theguardian.com,https://www.theguardian.com/world/2019/jun/13/...,-1,0,0,1,0,0
2,3,2019-04-30,Thejc,US army veteran arrested after planning terror...,A US army veteran has been arrested in souther...,https://www.thejc.com/news/us-news/us-army-vet...,Crimes and Arrests | Terrorism and Extremism |...,None,New Caledonia,www.thejc.com,https://www.thejc.com/news/us-news/us-army-vet...,0,0,0,0,0,0
3,4,2019-07-06,Express,Johnson calls on immigrants to learn English t...,The Tory leadership favourite made the remarks...,https://www.express.co.uk/news/politics/115003...,Politics | Hate Speech and Discrimination,None,United Kingdom,www.express.co.uk,https://www.express.co.uk/news/politics/115003...,-1,0,0,1,0,0
4,5,2019-02-01,Christiantoday,"In packed churches and secret masses, papal vi...",In Dubai's overflowing churches and Riyadh's s...,https://www.christiantoday.com/article/in-pack...,Religion,None,Saudi Arabia,www.christiantoday.com,https://www.christiantoday.com/article/in-pack...,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,496,2019-09-12,Dailymail,Australia presses Iran to release 3 citizens f...,"CANBERRA, Australia (AP) - Australia's foreign...",https://www.dailymail.co.uk/wires/ap/article-7...,Crimes and Arrests | Politics | Minorities and...,None,"Iran, Islamic Republic of",www.dailymail.co.uk,https://www.dailymail.co.uk/wires/ap/article-7...,0,0,0,0,0,0
496,497,2019-03-02,Dailymail,Bernie Sanders gets personal and calls Trump '...,branded\nas 'the most dangerous President in m...,https://www.dailymail.co.uk/news/article-67644...,Business and Economy | Terrorism and Extremism...,None,Poland,www.dailymail.co.uk,https://www.dailymail.co.uk/news/article-67644...,0,0,0,0,0,0
497,498,2019-08-19,Bbc,Kashmir schools re-open but students stay home,Some schools re-opened in Indian-administered ...,https://www.bbc.co.uk/news/world-asia-india-49...,Politics | Education | Minorities and Human Ri...,None,India,www.bbc.co.uk,https://www.bbc.co.uk/news/world-asia-india-49...,0,0,0,0,0,0
498,499,2019-07-25,Dailymail,"After 10 years of Boko Haram violence, Nigeria...","MAIDUGURI, Nigeria (AP) - Suicide bombings, ma...",https://www.dailymail.co.uk/wires/ap/article-7...,Business and Economy | Terrorism and Extremism,None,Nigeria,www.dailymail.co.uk,https://www.dailymail.co.uk/wires/ap/article-7...,1,0,0,1,1,0


In [35]:
[i for i in cursor.execute(sql_query)][0]

('2018-10-03', '2019-09-30')

In [36]:
from datetime import datetime

In [47]:
datetime.strptime('2018-10-03', '%Y-%m-%d').strftime('%Y-%m-%d')

'2018-10-03'

In [86]:
def build_query(selected_publisher, start_date, end_date, compared_publishers, topics):
    start_date = start_date.strftime('%Y-%m-%d')
    end_date = end_date.strftime('%Y-%m-%d')
    sql = f"SELECT * FROM articles WHERE (date_published >= '{start_date}' AND date_published <= '{end_date}')"

    publishers = [selected_publisher] + compared_publishers
    if len(publishers) > 0:
        publisher_sql = ' AND (' + ' OR '.join([f"publisher LIKE '%{i}%'" for i in publishers]) + ')'
        sql += publisher_sql

    if len(topics) > 0:
        topic_sql = ' AND (' + ' OR '.join([f"topic LIKE '%{i}%'" for i in topics]) + ')'
        sql += topic_sql
    
    return sql
    

In [87]:
sql_query = build_query('Bbc',
            datetime.strptime('2018-10-03', '%Y-%m-%d'),
            datetime.strptime('2019-01-01', '%Y-%m-%d'),
            ['Dailymail', 'Express', 'Independent'],
            [''],
            ['Immigration'])

sql_query

"SELECT * FROM articles WHERE (date_published >= '2018-10-03' AND date_published <= '2019-01-01') AND (publisher LIKE '%Bbc%') AND (topic LIKE '%Immigration%')"

In [88]:
pd.read_sql_query(sql_query, conn)

,id,date_published,publisher,title,text,article_url,topic,topic_list,location,main_site


In [70]:
pd.read_sql_query("SELECT * FROM articles WHERE (date_published >= '2018-10-03' AND date_published <= '2019-01-01')", conn)

,id,date_published,publisher,title,text,article_url,topic,topic_list,location,main_site
0,6,2018-11-18,Mirror,ISIS bride sisters held in Syria are part of U...,Two sisters who are both brides of men who die...,https://www.mirror.co.uk/news/uk-news/isis-bri...,Children and Women's Rights | Crimes and Arres...,None,Syrian Arab Republic,www.mirror.co.uk
1,17,2018-12-24,Dailymail,Turkey-backed fighters prepare to replace US f...,BEIRUT (AP) - Turkish-backed fighters said Mon...,https://www.dailymail.co.uk/wires/ap/article-6...,Immigration | Business and Economy | Terrorism...,None,Syrian Arab Republic,www.dailymail.co.uk
2,24,2018-10-19,Inews,Are you smarter than your kid? Take this quiz ...,Take the quiz:\nPick the correct definition of...,https://inews.co.uk/light-relief/quizzes/are-y...,Children and Women's Rights | Business and Eco...,None,United States,inews.co.uk
3,29,2018-10-29,Theguardian,Black Earth Rising recap: the finale – it has ...,Michaela Coel gives a magnificent performance ...,https://www.theguardian.com/tv-and-radio/2018/...,Business and Economy | Crimes and Arrests | Po...,None,Rwanda,www.theguardian.com
4,30,2018-12-17,News.Sky,Janbaz Tarin jailed for life for murdering ex-...,'They were let down by the authorities'\nA 21-...,https://news.sky.com/story/janbaz-tarin-man-ad...,Children and Women's Rights | Terrorism and Ex...,None,Afghanistan,news.sky.com
...,...,...,...,...,...,...,...,...,...,...
117,470,2018-10-05,Dailymail,Pakistan opposition leader Shahbaz Sharif arre...,Former Pakistani Prime Minister Nawaz Sharif a...,https://www.dailymail.co.uk/wires/afp/article-...,Crimes and Arrests | Politics | Immigration | ...,None,Pakistan,www.dailymail.co.uk
118,474,2018-12-22,Independent,The A-Z of Believing: S is for Sex | The Indep...,Song of Songs 7: 9-10,https://www.independent.co.uk/life-style/sex-s...,,None,None,www.independent.co.uk
119,477,2018-12-13,Independent,Government urged to adopt new definition of Is...,The government has been urged by more than 50 ...,https://www.independent.co.uk/news/uk/politics...,Children and Women's Rights | Politics | Hate ...,None,United Kingdom,www.independent.co.uk
120,481,2018-11-09,Dailymail,4 car bombs explode by hotel in Somalia's capi...,"MOGADISHU, Somalia (AP) - Four car bombs by Is...",https://www.dailymail.co.uk/wires/ap/article-6...,Terrorism and Extremism,None,Somalia,www.dailymail.co.uk


# LLM Based Content Generator

In [7]:
import openai
import requests
from dotenv import load_dotenv
import os

In [5]:
# Load the .env file
load_dotenv()

# Get the API key from the .env file
api_key = os.getenv("OPENAI_API_KEY")

In [ ]:
# Load the image file
with open("path_to_your_image.jpg", "rb") as image_file:
    image_data = image_file.read()

# Make the API call with the image and prompt
response = openai.ChatCompletion.create(
    model="gpt-4-vision",  # gpt-4 with vision capabilities
    messages=[
        {"role": "system", "content": "Describe the image in detail."}
    ],
    files=[
        {"file": image_data, "name": "image.jpg", "purpose": "fine-tune"}
    ]
)

# Extract and print the description
description = response.choices[0].message['content']
print(description)

In [8]:
class OpenAITextGenerator:
    def __init__(self):
        # Get the API key from the .env file
        self.api_key = os.getenv("OPENAI_API_KEY")
        openai.api_key = self.api_key
        self.messages = [{"role": "system",
                          "content": "You are a knowledgeable assistant in building reports."}]
    
    def generate_text(self, prompt):
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo", 
            messages=[
                {"role": "user", "content": prompt}
            ]
        )
        
        # Extract and return the generated response
        generated_text = response.choices[0].message['content']
        return generated_text


## Generate Methodology